# Проект

# Распознавание дорожных знаков

### Модель YOLO

In [10]:
pip install kagglehub opencv-python torch torchvision

In [2]:
import kagglehub

path_to_dataset = kagglehub.dataset_download("pkdarabi/cardetection")
print("Файлы датасета загружены по пути:", path_to_dataset)

100%|██████████| 99.8M/99.8M [00:03<00:00, 28.6MB/s]

Extracting model files...


Файлы датасета загружены по пути: C:\Users\1\.cache\kagglehub\datasets\pkdarabi\cardetection\versions\5


In [3]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.Collecting ultralytics
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)

  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.3
    Uninstalling pandas-1.1.3:
      Successfully uninstalled pandas-1.1.3
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.50.2
    Uninstalling tqdm-4.50.2:
      Successfully uni

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.26.1 requires fsspec>=2023.5.0, but you'll have fsspec 0.8.3 which is incompatible.


In [4]:
from ultralytics import YOLO
import cv2

#### Пробуем распознавать знаки с помощью предобученной модели

In [8]:
# Загрузка модели
model = YOLO('yolov8s.pt')

# Путь к видео
video_path = '20250608_154251.mp4'
cap = cv2.VideoCapture(video_path)

# Настройка вывода видео
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_annotated.mp4', fourcc, cap.get(5),
                      (int(cap.get(3)), int(cap.get(4))))

detected_signs = set()  # Для хранения уникальных знаков
frame_num = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_num += 1
    print(f"Кадр {frame_num}")

    # Используем модель
    results = model(frame)[0]

    for box in results.boxes:
        cls_id = int(box.cls[0])
        confidence = float(box.conf[0])
        if confidence < 0.5:
            continue

        name = model.names[cls_id]
        detected_signs.add(name)

        # Координаты
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        x1, y1, x2, y2 = xyxy

        # Рисуем
        label = f"{name} {confidence:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()

# Сохраняем список всех уникальных меток
with open('detected_signs.txt', 'w') as f:
    for label in sorted(detected_signs):
        f.write(f"{label}\n")

print("✅ Обработка завершена")
print("✔️ Найдено знаков:", len(detected_signs))
print("✍️ Сохранили в detected_signs.txt")

Кадр 1

0: 384x640 5 persons, 2 cars, 453.3ms
Speed: 66.4ms preprocess, 453.3ms inference, 24.9ms postprocess per image at shape (1, 3, 384, 640)
Кадр 2

0: 384x640 3 persons, 2 cars, 310.8ms
Speed: 7.0ms preprocess, 310.8ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
Кадр 3

0: 384x640 5 persons, 2 cars, 328.8ms
Speed: 7.6ms preprocess, 328.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
Кадр 4

0: 384x640 5 persons, 3 cars, 1 truck, 1 traffic light, 1 skateboard, 351.2ms
Speed: 5.2ms preprocess, 351.2ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)
Кадр 5

0: 384x640 7 persons, 3 cars, 343.5ms
Speed: 6.4ms preprocess, 343.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Кадр 6

0: 384x640 3 persons, 2 cars, 1 truck, 1 kite, 333.0ms
Speed: 5.3ms preprocess, 333.0ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Кадр 7

0: 384x640 4 persons, 2 cars, 332.3ms
Speed: 8.3ms preprocess, 332.3m

Кадр 56

0: 384x640 9 cars, 1 truck, 323.8ms
Speed: 6.9ms preprocess, 323.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Кадр 57

0: 384x640 9 cars, 1 bus, 390.9ms
Speed: 5.2ms preprocess, 390.9ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
Кадр 58

0: 384x640 9 cars, 384.4ms
Speed: 10.8ms preprocess, 384.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Кадр 59

0: 384x640 8 cars, 443.4ms
Speed: 8.1ms preprocess, 443.4ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
Кадр 60

0: 384x640 9 cars, 1 truck, 445.8ms
Speed: 5.3ms preprocess, 445.8ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
Кадр 61

0: 384x640 10 cars, 471.1ms
Speed: 6.3ms preprocess, 471.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
✅ Обработка завершена
✔️ Найдено знаков: 5
✍️ Сохранили в detected_signs.txt


На видео встречается знак ограничения скорости и знак пешеходного перехода. Но ни один из них не был распознан. 
Как оказалось, у предобученной модели из подходящих меток есть только светофор и знак "стоп".

Вывод: предобученная модель не подходит. Нужно обучать свою модель

#### Обучаем свою модель

In [13]:
!yolo task=detect \
  mode=train \
  model=yolov8s.pt \
  data=/kaggle/input/cardetection/car/data.yaml \
  epochs=30 \
  imgsz=640 \
  batch=16 \
  name=traffic-sign-trained

Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/cardetection/car/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=traffic-sign-trained2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

In [7]:
# Загрузка модели
model = YOLO('best.pt')

# Путь к видео
video_path = 'samplee.mp4'
cap = cv2.VideoCapture(video_path)

# Настройка вывода видео
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_annotated_3.mp4', fourcc, cap.get(5),
                      (int(cap.get(3)), int(cap.get(4))))

detected_signs = set()  # Для хранения уникальных знаков
frame_num = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_num += 1
    print(f"Кадр {frame_num}")

    # 🔍 Используем модель
    results = model(frame)[0]

    for box in results.boxes:
        cls_id = int(box.cls[0])
        confidence = float(box.conf[0])
        if confidence < 0.5:
            continue

        name = model.names[cls_id]
        detected_signs.add(name)

        # Координаты
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        x1, y1, x2, y2 = xyxy

        # Рисуем
        label = f"{name} {confidence:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()

# Сохраняем список всех уникальных меток
with open('detected_signs.txt', 'w') as f:
    for label in sorted(detected_signs):
        f.write(f"{label}\n")

print("✅ Обработка завершена")
print("✔️ Найдено знаков:", len(detected_signs))
print("✍️ Сохранили в detected_signs.txt")

Кадр 1

0: 384x640 (no detections), 389.7ms
Speed: 9.1ms preprocess, 389.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Кадр 2

0: 384x640 (no detections), 329.2ms
Speed: 7.0ms preprocess, 329.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Кадр 3

0: 384x640 (no detections), 305.8ms
Speed: 4.9ms preprocess, 305.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Кадр 4

0: 384x640 (no detections), 347.7ms
Speed: 5.4ms preprocess, 347.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Кадр 5

0: 384x640 (no detections), 295.2ms
Speed: 6.2ms preprocess, 295.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Кадр 6

0: 384x640 (no detections), 300.4ms
Speed: 4.7ms preprocess, 300.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Кадр 7

0: 384x640 (no detections), 389.6ms
Speed: 5.4ms preprocess, 389.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Кадр 8

0: 384x640 1 Red Light, 1 Stop, 317.2ms
Speed: 4.6ms preprocess, 317.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Кадр 59

0: 384x640 1 Red Light, 1 Stop, 317.8ms
Speed: 3.8ms preprocess, 317.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Кадр 60

0: 384x640 1 Red Light, 1 Stop, 339.1ms
Speed: 5.5ms preprocess, 339.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Кадр 61

0: 384x640 1 Red Light, 1 Stop, 303.8ms
Speed: 6.1ms preprocess, 303.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Кадр 62

0: 384x640 1 Red Light, 1 Stop, 343.9ms
Speed: 6.1ms preprocess, 343.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Кадр 63

0: 384x640 1 Red Light, 1 Stop, 329.4ms
Speed: 3.8ms preprocess, 329.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Кадр 64

0: 384x640 1 Red Light, 1 Stop, 363.8ms
Speed: 4.6ms preprocess, 363.8ms inference, 1.6ms postprocess per image at sha

Кадр 114

0: 384x640 1 Red Light, 1 Stop, 282.9ms
Speed: 4.3ms preprocess, 282.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Кадр 115

0: 384x640 1 Red Light, 1 Stop, 303.8ms
Speed: 3.7ms preprocess, 303.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Кадр 116

0: 384x640 1 Red Light, 1 Stop, 304.0ms
Speed: 4.2ms preprocess, 304.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Кадр 117

0: 384x640 1 Red Light, 1 Stop, 287.2ms
Speed: 4.7ms preprocess, 287.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Кадр 118

0: 384x640 1 Red Light, 1 Stop, 286.7ms
Speed: 3.8ms preprocess, 286.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Кадр 119

0: 384x640 1 Red Light, 1 Stop, 282.6ms
Speed: 6.4ms preprocess, 282.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Кадр 120

0: 384x640 1 Red Light, 1 Stop, 280.3ms
Speed: 5.1ms preprocess, 280.3ms inference, 1.6ms postprocess 

Модель корректно распознала знаки на видео.

Выведем отдельно метрики модели

In [24]:
!yolo task=detect \
  mode=val \
  model=runs/detect/traffic-sign-trained2/weights/best.pt \
  data=/kaggle/input/cardetection/car/data.yaml \
  split=test

Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,131,389 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3.8±1.6 MB/s, size: 20.7 KB)
val: Scanning /kaggle/input/cardetection/car/test/labels... 638 images, 1 backgrounds, 0 corrupt: 100% 638/638 [00:07<00:00, 80.12it/s]
WARNING ⚠️ val: Cache directory /kaggle/input/cardetection/car/test is not writeable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 40/40 [00:08<00:00,  4.50it/s]
                   all        638        770       0.93      0.903      0.947      0.811
           Green Light         77        110      0.878      0.851      0.913       0.58
             Red Light         71         94       0.85      0.724      0.812      0.491
        Speed Limit 10          2          3      0.633      0.667      0.775      0.637
       Speed Limit 100         45  